In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score
import lightgbm
import xgboost
import joblib
import json
from datetime import datetime
from feature_engineering import fe  # Import کردن تابع fe از فایل feature1.py که داده‌های ویژگی را پردازش می‌کند
from get_data import get_data  # Import کردن تابع get_data برای دریافت داده‌ها
from pre_inf import prepare_data, create_target, X_y_split


df = get_data()  
print('Data fetched...')
print(f'Total records fetched: {len(df)}')


df.to_csv(r'D:\Desktop\Time\all_data.csv', index=False)


df = pd.read_csv(r'D:\Desktop\Time\all_data.csv')
d_train = fe(df, train=False)


with open(r'D:\Desktop\Time\xgb_hyperparameter.txt', 'r') as xgb_file:
    best_xgb = json.load(xgb_file)
with open(r'D:\Desktop\Time\lgbm_hyperparameter.txt', 'r') as lgbm_file:
    best_lgb = json.load(lgbm_file)


up_bound = 1.07
low_band = 1.0
d_train = prepare_data(d_train) 
d_train = create_target(d_train, upper_bound=up_bound, lower_bound=low_band)  # ایجاد هدف
X, y = X_y_split(d_train) 


opt_model = joblib.load(r'D:\Desktop\Time\model.pkl')


d = fe(df, train=False)  
d = d.sort_values('date_m')
d.to_csv(r'D:\Desktop\Time\inference_dataset_modular_test.csv', index=False)


d = pd.read_csv(r'D:\Desktop\Time\inference_dataset_modular_test.csv')
d = prepare_data(d) 


d['date_m'] = pd.to_datetime(d['date_m'])
today_date = d['date_m'].max() 
today_records = d[d['date_m'] == today_date].drop(columns=['date_m'])


pred_proba = opt_model.predict_proba(today_records)  


LE = LabelEncoder()
share = pd.read_csv('new_info.csv')[['کد نماد', 'نماد', 'نام لاتین', 'نماد.1']]
share.columns = ['en_symbol_12_digit_code', 'symbol', 'en_symbol_5_digit_code', 'fa_symbol']
LE.classes_ = np.load(r'D:\Desktop\Time\classes_en_symbol_12_digit_code.npy', allow_pickle=True)
today_records['en_symbol_12_digit_code'] = LE.inverse_transform(today_records['en_symbol_12_digit_code'].values)
results_new = pd.DataFrame({
    'date': today_date,
    'first_price': today_records['first_price'],
    'max_price': today_records['max_price'],
    'min_price': today_records['min_price'],
    'last_price': today_records['last_price'],
    'en_symbol_12_digit_code': today_records['en_symbol_12_digit_code'],
    'prediction': pred_proba[:, 1]  
})


results_new = pd.merge(results_new, share[['en_symbol_12_digit_code', 'en_symbol_5_digit_code', 'fa_symbol']], 
                       on='en_symbol_12_digit_code', how='left')
results_new.to_csv(r'D:\Desktop\Time\results_new.csv', index=False, encoding='utf-8')


r = pd.read_csv(r'D:\Desktop\Time\results_new.csv')
r_sorted = r.sort_values(by='prediction', ascending=False)  # مرتب‌سازی براساس احتمال‌ها
print(r_sorted[['date', 'en_symbol_12_digit_code', 'fa_symbol', 'first_price', 'max_price', 'min_price', 'last_price', 'prediction']])



C:\Users\USER\ma\feature_engineering.py:21: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  macd_ind = df.groupby('en_symbol_12_digit_code').apply(lambda x: pd.concat([x['date_m'], x['en_symbol_12_digit_code'], macd(x['last_price'])], axis=1)).reset_index(drop=True)
C:\Users\USER\ma\feature_engineering.py:22: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence

ستون‌های نهایی داده‌ها:
['date_m', 'first_price', 'max_price', 'min_price', 'end_price', 'trade_value', 'trade_volume', 'trade_count', 'yesterday_price', 'last_price', 'date_sh', 'person_buy_count', 'Legal_buy_count', 'person_sell_count', 'Legal_sell_count', 'person_buy_volume', 'Legal_buy_volume', 'person_sell_volume', 'Legal_sell_volume', 'person_buy_value', 'Legal_buy_value', 'person_sell_value', 'Legal_sell_value', 'fa_symbol', 'en_symbol_12_digit_code', 'sub_industry_code', 'industry_code', 'index', 'en_symbol', 'en_name', 'fa_name', 'person_buy_avg_price', 'person_sell_avg_price', 'Legal_buy_avg_price', 'Legal_sell_avg_price', 'person_ownership_change', 'Legal_ownership_change', 'new_trade_volume', 'new_Legal_buy_volume', 'new_person_buy_volume', 'new_first_price', 'new_last_price', 'new_min_price', 'new_max_price', 'new_end_price', 'new_person_buy_avg_price', 'new_Legal_buy_avg_price', 'new_person_sell_avg_price', 'new_Legal_sell_avg_price', 'legal_enter_money', 'person_enter_mo

0it [06:01, ?it/s]


ConnectionError: HTTPConnectionPool(host='old.tsetmc.com', port=80): Max retries exceeded with url: /tsev2/data/clienttype.aspx?i=59612098290740355 (Caused by ReadTimeoutError("HTTPConnectionPool(host='old.tsetmc.com', port=80): Read timed out. (read timeout=8)"))